# The Easiest Way to Deploy Your ML/DL Models in 2022: Streamlit + BentoML + DagsHub
## Deploy models as lightweight APIs with a user-friendly interface
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/photo/blue-red-and-black-abstract-painting-2130475/'>Steve Johnson</a>
    </strong>
</figcaption>


### Introduction

### What problem are we solving?

### What tools will we be using?

### Step 1: Save the best model as a bento

### Step 2: Build the service

### Step 3: Containerize and deploy to Heroku

### Step 4: Build the UI with Streamlit

### Conclusion